# Import text from the JSON file

In [67]:
import json 
import pandas as pd

In [70]:
with open("your own JSON file path") as f:
    discussions = json.load(f)

In [71]:
def read_comment(comment, result):
    author = comment['user']['name']
    
    replies = comment['comments']
    for reply in replies:
        replier = reply['user']['name']
        result.append((author, replier))
        
        read_comment(reply, result)

from_to_list = []
for discussion in discussions:
    author = discussion['user']['name']
    from_to_list.append((author, None))
    read_comment(discussion, from_to_list)

    
#print(from_to_list)


            
             
        
            
            
        
        

In [72]:
from_list = []
to_list = []
for pair in from_to_list:
    from_list.append(pair[0])
    to_list.append(pair[1])

# print(from_list)
# print(to_list)

social_network_data_frame = pd.DataFrame(
    {'from': from_list,
     'to': to_list
    })
#print(social_network_data_frame.head())